# ECE HW 1
# NetID = "somani4"
# Name = "Akhilesh Somani"

# Part A: Parsing

In [1]:
# read in pf.log
with open("pf.log",'r') as f:
    lines = f.readlines()

In [2]:
lines = [x.strip() for x in lines]
# strip all lines of whitespace

In [3]:
# parse each line into your data structure
my_dict={}
z_backtrace=[]

for line in lines:
    
    x=line.split(":")
    if len(x)==7:
        z_info=(x[0],x[1],x[2],int(x[3],16),x[4],x[5],x[6])
        z_backtrace=[]
    else:
        second_half=line.split("+")
        temp=second_half[1].split("/")
        z_backtrace_temp=[second_half[0].lstrip('<'),int(temp[1].rstrip('>'),16), int(temp[0],16)]
        z_backtrace.append(z_backtrace_temp)
    
    
    my_dict[z_info]=z_backtrace    


In [4]:
# write your data structure out to pf.csv
# include an index column with unique indices for each page fault

with open('pf.csv', 'w') as f:
    # write header line
    f.write('\t'.join(['index', 'time', 'proc_name', 'pid', 'pfaddr', 'rw', 'major_minor', 'resolve_time', 'lib', 'addr', 'offset']))
    f.write('\n')
    
    data_entry=[]
    for i, key in enumerate (my_dict.keys()):
        for value in my_dict[key]:
            data_entry.append([i+1]+list(key)+value)
            
    for entry in data_entry:
        entry[0]=str(entry[0])
        entry[4]=str(entry[4])
        entry[-1]=str(entry[-1])
        entry[-2]=str(entry[-2])
    
    
    for entry in data_entry:
        f.write('\t'.join(entry))
        f.write('\n')




PermissionError: [Errno 13] Permission denied: 'pf.csv'

# Part B: Analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mlab
import matplotlib.pyplot as plt

In [ ]:
# relevant options when reading in pf.csv: sep, parse_dates
df = pd.read_csv('pf_trial1.csv', sep = '\t', parse_dates = True)


In [ ]:
df['time']=pd.to_datetime(df['time'],unit='ms')

# Question [B] (a)

In [ ]:
print("The time that the data ranges is from {0} to {1}".format(df['time'].min(),df['time'].max()) )

# Question [B] (b)

In [ ]:
print("Number of unique processes run are: {0}".format(len(df.groupby('proc_name'))))

In [ ]:
my_df = df.set_index('index')
my_df = my_df.loc[~my_df.index.duplicated(keep='first')]

In [ ]:
print(my_df['proc_name'].value_counts(sort=False, ascending=False))

# Question [B] (c)

In [ ]:
my_df['major_minor'].value_counts(sort=False, ascending=False)

In [ ]:
y_values1=my_df['major_minor'].value_counts()['major']

y_values2=my_df['major_minor'].value_counts()['minor']
    
column_names=my_df.columns
column_names=list(column_names)
column_names.remove('major_minor')

x_label=np.arange(len(column_names))

fig, ax = plt.subplots(figsize=(20, 10))

bar1=plt.bar(x_label, y_values1, width = 0.4, bottom = None, align='edge',color='b',label='Major Fault')
bar2=plt.bar(x_label, y_values2, width = -0.4, bottom = None, align='edge',color='r',label='Minor Fault')


plt.xlabel('Other information parameters about the page fault',fontsize=25)
plt.ylabel('Number of Page Faults', fontsize=25)
plt.xticks(x_label,column_names)
plt.legend()
ax.tick_params(axis='both', which='major', labelsize=20)

# Question [B] (d)

In [ ]:
time_major = [my_df.at[row+1,'resolve_time'] for row in range(len(my_df)) if my_df.at[row+1,'major_minor'] == 'major']

In [ ]:
fig,ax =plt.subplots(figsize=(20,10))
plt.hist(time_major, bins=200) 
plt.xlabel('Resolve Time (ms)',fontsize=25)
plt.ylabel('Number', fontsize=25)
plt.title('Resolve Time for Major Faults',fontsize=25)
ax.tick_params(axis='both', which='major', labelsize=20)

In [ ]:
time_minor = [my_df.at[row+1,'resolve_time'] for row in range(len(my_df)) if my_df.at[row+1,'major_minor'] == 'minor']

In [ ]:
fig,ax =plt.subplots(figsize=(20,10))
plt.hist(time_minor, bins=50) 
plt.xlabel('Resolve Time (ms)',fontsize=25)
plt.ylabel('Number', fontsize=25)
plt.title('Resolve Time for Minor Faults',fontsize=25)
ax.tick_params(axis='both', which='major', labelsize=20)

In [ ]:
process_names= my_df['proc_name'].unique()

In [ ]:
mean_table_major=my_df.loc[(my_df['major_minor']=='major')].groupby(my_df.proc_name,sort=False).mean()
std_table_major=my_df.loc[(my_df['major_minor']=='major')].groupby(my_df.proc_name,sort=False).std()

mean_table_minor=my_df.loc[(my_df['major_minor']=='minor')].groupby(my_df.proc_name,sort=False).mean()
std_table_minor=my_df.loc[(my_df['major_minor']=='minor')].groupby(my_df.proc_name,sort=False).std()

process_names= my_df['proc_name'].unique()

In [ ]:
for process in process_names:
    print("***********For Process: {0}, the results are: \n".format(process))
    print("Major Faults: \n (1) Mean: {0} ms \n (2) Standard Deviation: {1} ms \n \n".format(mean_table_major.loc[process,'resolve_time'], std_table_major.loc[process,'resolve_time']))
    print("Minor Faults: \n (1) Mean: {0} ms \n (2) Standard Deviation: {1} ms \n \n".format(mean_table_minor.loc[process,'resolve_time'], std_table_minor.loc[process,'resolve_time']))

# Part C: Making Predictions

# Question [C] (a)

In [ ]:
Priors=my_df['proc_name'].value_counts(sort=False)/my_df['proc_name'].value_counts(sort=False, ascending=False).sum()
Priors

# Question [C] (b)

$$ P (C = process | Type = Major Fault) ∝ P (Type = Major Fault | C = process)*P(C=process) $$

In [ ]:
z=my_df.groupby(['proc_name','major_minor']).count()
p_major=[]
#p_minor=[]

for process in process_names:
    p_major.append((z.loc[(process,'major'),'time']/my_df['proc_name'].value_counts()[process])*Priors[process])
    #p_minor.append((z.loc[(process,'minor'),'time']/my_df['proc_name'].value_counts()[process])*Priors[process])

#MAP RULE
max_major=max(p_major)
#max_min=max(p_minor)

i_major = [i for i in range(len(p_major)) if p_major[i]==max_major]

#[i_minor for i_minor in range(len(p_minor)) if p_minor[i]==max_minor]

print("Given the fault was major, it was most likely caused by process: {0}".format(process_names[i_major[0]]))

# Question [C] (c)

$$ P (C = process | Access = Read) ∝ P (Access = Read | C = process)*P(C=process) $$

In [ ]:
z=my_df.groupby(['proc_name','rw']).count()
p_r=[]

for process in process_names:
    p_r.append((z.loc[(process,'R'),'time']/my_df['proc_name'].value_counts()[process])*Priors[process])
    
#MAP RULE
max_r=max(p_r)

i_r = [i for i in range(len(p_r)) if p_r[i]==max_r]

print("Given the fault was due to read access, it was most likely caused by process: {0}".format(process_names[i_r[0]]))